# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import folium
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,12.34,64,97,0.65,RU,1691971863
1,1,west island,-12.1568,96.8225,23.99,78,75,7.72,CC,1691971863
2,2,holyhead,53.3062,-4.6321,16.10,88,100,5.66,GB,1691971863
3,3,maintirano,-18.0667,44.0167,22.25,81,27,4.08,MG,1691971863
4,4,bethel,41.3712,-73.4140,27.17,71,0,2.06,US,1691971864


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
# Create a map centered around a specific location (you can choose a lat and lon)
m = folium.Map(location=[18.38, 42.4], zoom_start=2)

# Add markers for each city
for index, row in city_data_df.iterrows():
    popupInfo = f"Lat: {row['Lat']}<br>Lng: {row['Lng']}<br>City: {row['City']}<br>Humidity: {row['Humidity']}"
    
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 15,
        popup=folium.Popup(html=popupInfo, max_width=200), #Click on the marker to reaveal this info. The hover option didint like non standard characters
        fill = True,
        fill_opacity = 1
    ).add_to(m)

# Display the map
m


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down the city_data_df DataFrame to find your ideal weather condition

# Define criteria for narrowing down cities
ideal_criteria = (city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27) & \
                 (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] == 0)

# Use the criteria to filter the DataFrame
ideal_cities_df = city_data_df[ideal_criteria]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,jamestown,42.0970,-79.2353,21.92,73,0,2.06,US,1691971848
43,43,xifeng,42.7372,124.7222,22.74,85,0,1.24,CN,1691971871
44,44,darnah,32.7670,22.6367,25.12,65,0,4.59,LY,1691971871
207,207,matoury,4.8483,-52.3317,25.98,89,0,0.00,GF,1691971902
224,224,sferracavallo,42.7260,12.0956,22.12,81,0,0.71,IT,1691971904
262,262,southbury,41.4815,-73.2132,26.65,73,0,2.06,US,1691971909
276,276,gharyan,32.1722,13.0203,20.09,61,0,3.10,LY,1691971911
312,312,remire-montjoly,4.9167,-52.2667,26.02,89,0,0.00,GF,1691971916
325,325,binzhou,37.3667,118.0167,25.86,72,0,1.07,CN,1691971917
359,359,south boston,42.3334,-71.0495,26.06,71,0,2.57,US,1691971925


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store specific columns
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = ideal_cities_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,jamestown,US,42.0970,-79.2353,73,
43,xifeng,CN,42.7372,124.7222,85,
44,darnah,LY,32.7670,22.6367,65,
207,matoury,GF,4.8483,-52.3317,89,
224,sferracavallo,IT,42.7260,12.0956,81,
262,southbury,US,41.4815,-73.2132,73,
276,gharyan,LY,32.1722,13.0203,61,
312,remire-montjoly,GF,4.9167,-52.2667,89,
325,binzhou,CN,37.3667,118.0167,72,
359,south boston,US,42.3334,-71.0495,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000  # in meters
params = {
    "type": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key  # Make sure you have imported your API key correctly
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
xifeng - nearest hotel: No hotel found
darnah - nearest hotel: فندق الفردوس
matoury - nearest hotel: La Marmotte
sferracavallo - nearest hotel: Hotel Oasi dei discepoli
southbury - nearest hotel: The Milestone Inn
gharyan - nearest hotel: فندق الرابطة
remire-montjoly - nearest hotel: Complexe Belova
binzhou - nearest hotel: No hotel found
south boston - nearest hotel: Quarters on Dots
callao - nearest hotel: Casa Ronald
tete - nearest hotel: Zambezi Hotel
krymsk - nearest hotel: Крымск
shevchenko - nearest hotel: Premier Hotel Abri
hami - nearest hotel: Хами
bastia - nearest hotel: Adonis Saint Florent
ivanivka - nearest hotel: No hotel found
giado - nearest hotel: فندق جادو السياحي
astara - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
29,jamestown,US,42.0970,-79.2353,73,DoubleTree Jamestown
43,xifeng,CN,42.7372,124.7222,85,No hotel found
44,darnah,LY,32.7670,22.6367,65,فندق الفردوس
207,matoury,GF,4.8483,-52.3317,89,La Marmotte
224,sferracavallo,IT,42.7260,12.0956,81,Hotel Oasi dei discepoli
262,southbury,US,41.4815,-73.2132,73,The Milestone Inn
276,gharyan,LY,32.1722,13.0203,61,فندق الرابطة
312,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
325,binzhou,CN,37.3667,118.0167,72,No hotel found
359,south boston,US,42.3334,-71.0495,71,Quarters on Dots


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
m = folium.Map(location=[18.38, 42.4], zoom_start=2)

# Add markers for each city
for index, row in hotel_df.iterrows():
    tooltip = f"Lat: {row['Lat']}<br>Lng: {row['Lng']}<br>Country: {row['Country']}<br>City: \
    {row['City']}<br>Hotel: {row['Hotel Name']}<br>Humidity: {row['Humidity']}"
    
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 15,
        tooltip= tooltip,
        fill = True,
        fill_opacity = 1
    ).add_to(m)

m